### Самостоятельная работа

1. Часть 1. Работа с GeoPandas. 
    1. Выгрузить из google drive файл  
(https://drive.google.com/drive/folders/10ZxFd0dQHYU-BINLBi3dxbZZ5_Mp5qBj)  
в `geopandas geodataframe`
    1. Проверить содержимое с помощью метода `info`
    1. Визуализировать получившийся `geodataframe`
1. Часть 2. Работа с pydantic и geojson-pydantic. 
    1. Опираясь на результат метода `info` составить  
`geojson-pydantic` схему для валидации входных данных:
    - составить схему для валидации свойст (properties).  
    Названия полей должны использовать стилистику python  
    относительно названия переменных (см. PEP8)
    - В качестве геометрий выбрать Point 
1. Часть 3. Валидация данных. 
    1. С помощью модуля `requests` загрузить тот же самый json файл  
и проверить его с помощью составленной `geojson pydantic` схемы.


### Подготовка

In [1]:
# установка библиотек
!pip install folium -U
!pip install geopandas mapclassify osmnx
!pip install pydantic
!pip install geojson-pydantic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from traceback import print_exc
from typing import List, Dict, Union, Optional # Для типизации
import requests

import pandas as pd
import geopandas as gpd

from pydantic import BaseModel, Field # Валидация JSON формата
from pydantic.error_wrappers import ValidationError # Обработка ошибок при валидации
from geojson_pydantic import FeatureCollection, Point, Polygon # Валидация GeoJSON формата

In [3]:
def get_google_drive_download_url(url: str) -> str:
    """
    Функция возвращает ссылку для скачивания с google drive.
     Например: "https://drive.google.com/file/d/1MWGOuqV76e0ubQOg8Ke0KTU3yGpkprHp/view?usp=sharing" ->
     "https://drive.google.com/uc?export=download&id=1MWGOuqV76e0ubQOg8Ke0KTU3yGpkprHp"
    """
    
    drive_id = url.split("/")[5]
    return f"https://drive.google.com/uc?export=download&id={drive_id}"

### Часть 1

In [12]:
# Выгружаем один из файлов на диске: "https://drive.google.com/drive/folders/10ZxFd0dQHYU-BINLBi3dxbZZ5_Mp5qBj"
google_drive_url = "https://drive.google.com/file/d/1dBsNFkKI0L2Gs0FqesygDLPNYrRrZXyq/view?usp=sharing"
download_url = get_google_drive_download_url(google_drive_url)

In [13]:
gdf = gpd.read_file(download_url) # Загружаем данные в geopandas
gdf.head()

,Name,City function,Type,Coordinates y,Coordinates x,Coordinates xy,geometry
0,Бассейн СКА,Спорт,Бассейн,59.936746,30.252034,"30.25187133629281,59.93674916320182",POINT (30.25203 59.93675)
1,Грифон,Спорт,Бассейн,59.927734,30.326837,"30.3267591,59.9276414",POINT (30.32684 59.92773)
2,СШ №2,Спорт,Бассейн,59.955728,30.427304,"30.42728139697902,59.95578571510781",POINT (30.42730 59.95573)
3,"детский бассейн ""Утёнок"" (ГБОУ СОШ № 21)",Спорт,Бассейн,59.940078,30.283987,"30.2841716,59.9402942",POINT (30.28399 59.94008)
4,Сокол,Спорт,Бассейн,59.878184,30.308888,"30.30896040183802,59.87818302977694",POINT (30.30889 59.87818)


In [14]:
# Проверяем содержимое
gdf.info() 

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Name            71 non-null     object  
 1   City function   71 non-null     object  
 2   Type            71 non-null     object  
 3   Coordinates y   71 non-null     float64 
 4   Coordinates x   71 non-null     float64 
 5   Coordinates xy  71 non-null     object  
 6   geometry        71 non-null     geometry
dtypes: float64(2), geometry(1), object(4)
memory usage: 4.0+ KB


In [38]:
# Визуализируем данные
gdf.explore()

### Часть 2

Опираясь на результат метода info составить
geojson-pydantic схему для валидации входных данных:

In [17]:
# Составим схему для валидации данных
class MyProperties(BaseModel):
    Name : str
    CityFunction: str = Field(..., alias="City function")
    Type : str
    Coordinates_y: float = Field(..., alias="Coordinates y")
    Coordinates_x: float = Field(..., alias="Coordinates x")
    Coordinates_xy: str = Field(..., alias="Coordinates xy")

### Часть 3

С помощью модуля requests загрузить тот же самый json файл
и проверить его с помощью составленной geojson pydantic схемы.

In [31]:
# Загружаем данные в формате GeoJSON
resp = requests.get(download_url)
data_json = resp.json()

In [34]:
# Проанализируем полученный GeoJSON
print(data_json.keys())
print(data_json['type'])
print(data_json['name'])
print(data_json['crs'])
print(data_json['features'][0])

dict_keys(['type', 'name', 'crs', 'features'])
FeatureCollection
swimming_pools
{'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}}
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [30.252034, 59.936746]}, 'properties': {'Name': 'Бассейн СКА', 'City function': 'Спорт', 'Type': 'Бассейн', 'Coordinates y': 59.936746, 'Coordinates x': 30.252034, 'Coordinates xy': '30.25187133629281,59.93674916320182'}}


In [32]:
MyFeatureCollection = FeatureCollection[Point, MyProperties]  # Формируем свое видение geojson
feature_collection = MyFeatureCollection(**data_json) # Валидируем данные

In [37]:
# Посмотрим на полученные данные после валидации
print(feature_collection.dict().keys())  # Получить словарь из pydantic объекта
print(feature_collection.dict()['features'][0])

dict_keys(['type', 'features', 'bbox'])
{'type': 'Feature', 'geometry': {'coordinates': (30.252034, 59.936746), 'type': 'Point'}, 'properties': {'Name': 'Бассейн СКА', 'CityFunction': 'Спорт', 'Type': 'Бассейн', 'Coordinates_y': 59.936746, 'Coordinates_x': 30.252034, 'Coordinates_xy': '30.25187133629281,59.93674916320182'}, 'id': None, 'bbox': None}
